In [6]:
import sys
from pathlib import Path

In [7]:
src_path=Path("../src").resolve()
sys.path.append(str(src_path))

In [8]:
from sqlmodel import Session, select
from api.db.session import engine # type: ignore
from api.events.models import EventModel # type: ignore

In [9]:
from timescaledb.hyperfunctions import time_bucket
from pprint import pprint
from sqlalchemy import func
from datetime import datetime, timedelta, timezone

with Session(engine) as session:
    bucket=time_bucket("1 day", EventModel.time)
    pages=["/home","/about","/contact","/pages","pricing"]
    start=datetime.now(timezone.utc)-timedelta(hours=1)
    finish=datetime.now(timezone.utc) + timedelta(hours=1)
    statement = (
        select(
            bucket,
            EventModel.page,
            func.count()
            )
            .where(
                EventModel.time>start,
                EventModel.time<=finish,
                EventModel.page.in_(pages),
            )
            .group_by(
                bucket,
                EventModel.page,
            )
            .order_by(
                EventModel.page,
                bucket,
                )
        )
    compiled_query=statement.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    results = session.exec(statement).fetchall()
    pprint(results)

SELECT time_bucket('1 day'::interval, eventmodel.time) AS time_bucket_1, eventmodel.page, count(*) AS count_1 
FROM eventmodel 
WHERE eventmodel.time > '2025-03-27 09:37:02.304233+00:00' AND eventmodel.time <= '2025-03-27 11:37:02.304254+00:00' AND eventmodel.page IN ('/home', '/about', '/contact', '/pages', 'pricing') GROUP BY time_bucket('1 day'::interval, eventmodel.time), eventmodel.page ORDER BY eventmodel.page, time_bucket('1 day'::interval, eventmodel.time)
[]
